# 2. Inception-v3
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mcqueg/cnn-alzheimers/blob/main/notebooks/2-inception-v3.ipynb)  


The first model architecture we will explore for classifying the images is the [Inception-v3](https://arxiv.org/abs/1512.00567) architecture. The model architecture and pretrained weights from the ImageNet dataset can easily be downloaded and compiled from [Tensorflow and Keras](https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3) for transfer learning applications.

The process of transfer learning for ConvNets includes:

1. Getting only the convolutional layers and associated pretrained weights of the model.
2. Freeze the weights of the pretrained convolutional layers
3. Attach Dense connected layers to the top
4. Train Dense top layers on the desired task. 


In [12]:
import os
# set correct local working directory and useful paths
ROOT = '/Users/garrettmccue/projects/cnn-alzheimers'
# ROOT = 'cnn-alzheimers'
data_path = '/data/processed'
weights_path = '/models/raw'

os.chdir(ROOT)
os.getcwd()

'/Users/garrettmccue/projects/cnn-alzheimers'

Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/tensorflow/keras-applications/inception_v3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /Users/garrettmccue/projects/cnn-alzheimers/models/raw/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2022-07-21 22:30:19--  https://storage.googleapis.com/tensorflow/keras-applications/inception_v3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.190.144, 172.217.4.48, 142.250.191.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.190.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/octet-stream]
Saving to: ‘/Users/garrettmccue/projects/cnn-alzheimers/models/raw/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/Users/garrettmccue 100%[===================>]  83.84M  37.3MB/s    in 2.2s    

2022-07-21 22:30:21 (37.3 MB/s) - ‘/Users/garrettmccue/projects/cnn-alzheimers/models/raw/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [3]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

# set the downloaded weights file to a variable
local_weights_path = f'{weights_path}/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Initialize base model from tensorflow.keras
# Set the input shape and remove dense layers (top), to match our weights file
# our image shapes are (256x256)
pre_trained_model = InceptionV3(input_shape = (256, 256, 3),
                                include_top= False,
                                weights = None)

# load pretrained weights
pre_trained_model.load_weights(local_weights_path)

# freeze weights of each layer
for layer in pre_trained_model.layers:
    layer.trainable = False


2022-07-21 22:30:30.974216: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Since the model is very large the later layers tend to be very specialized, and will most likely not generalize well to our task, we are only going to use up to layer `mixed_8`. If we feel it needs to be changed we can make adjsutments later. Now the desired last layer can be set in the model.

In [4]:
# choose last layer from the pretrained model to feed into the dense layers
last_layer = pre_trained_model.get_layer('mixed8')
print(f'Last layer output shape: {last_layer.output_shape}')
last_output = last_layer.output

Last layer output shape: (None, 6, 6, 1280)


now we can add dense layers to be trained for classification. Dropout can also be added for regularization and avoid overfitting. 

In [5]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# add a fully connected layer
x = layers.Dense(1024, activation='relu')(x)
# add the dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# add a final layer with softmax for classification
x = layers.Dense(5, activation='softmax')(x)

# append the created dense network to the pre_trained_model
model = Model(pre_trained_model.input, x)

# print model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

The code to initialize the pretrained model and append a dense layer can be formated in a function within the inception-v3.py module to return a compiled model with specified training parameters. 

In [6]:
def build_inception_v3(input_shape, class_num, last_layer, lr, dropout, loss, metrics, print_summary=False):
    '''
    Purpose: 
        Initializes and compiles an inceptionv3 model with pretrained weights from ImageNet up \
         to the specified last layer using the passed training parameter arguments. Dense network \
         appended has a Dense layer of 1024 and droput layer as well as a classification \
         layer with softmax activation. 
    Parameters:
        input_shape - tuple - must be 3 channels i.e. (256, 256, 3)
        class_num - int - specified number of classes to set the shape of the output layer (softmax)
        last_layer - str - specified layer to use as the input layer to the dense network. \
                            see model.summary() for architecture
        lr - float - learning rate for defualt ADAM optimizer
        dropout - float - dropout to be applied after the Dense(1024) layer
        loss - str - specified loss to use
        metrics - list - str - list of metrics to use i.e. metrics = ['accuracy', 'loss']
        print_summary - bool - defualt = False; prints the model summary along with the model
    Returns:
        model - compiled model 
    '''
    # -- LOAD WEIGHTS --

    # set the downloaded weights file to a variable
    local_weights_path = f'{weights_path}/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

    # Initialize base model from tensorflow.keras
    # Set the input shape and remove dense layers (top), to match our weights file
    # our image shapes are (256x256)
    pre_trained_model = InceptionV3(input_shape = input_shape,
                                    include_top= False,
                                    weights = None)

    # load pretrained weights
    print(f'Loading Inception-v3 weights from {local_weights_path}...')
    pre_trained_model.load_weights(local_weights_path)

    # -- FORMATTING PRE-TRAINED MODEL -- 

    # freeze weights of each layer
    for layer in pre_trained_model.layers:
        layer.trainable = False

    print(f'Setting last layer as: {last_layer}')
    # choose last layer from the pretrained model to feed into the dense layers
    last_layer = pre_trained_model.get_layer('mixed8')
    print(f'Last layer output shape: {last_layer.output_shape}')
    last_output = last_layer.output
    
    # ---  ADDING DENSE LAYER ---

    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)
    # add a fully connected layer
    x = layers.Dense(1024, activation='relu')(x)
    # add the dropout rate of 0.2
    x = layers.Dropout(dropout)(x)
    # add a final layer with softmax for classification
    x = layers.Dense(class_num, activation='softmax')(x)

    # append the created dense network to the pre_trained_model
    model = Model(pre_trained_model.input, x)

   
    # -- COMPILE MODEL --
    model.compile(optimizer = Adam(learning_rate=lr),
                  loss = loss,
                  metrics = metrics)

    if print_summary:
        model.summary()
    
    return model

The build_inception_v3() function above can be tested from the models/inception-v3.py module.

In [7]:
os.getcwd()

'/Users/garrettmccue/projects/cnn-alzheimers'

In [8]:
from src.models.inception_v3 import build_inception_v3

INPUT_SHAPE = (256, 256, 3)
CLASS_NUM = 5
LAST_LAYER = 'mixed8'

v3_model = build_inception_v3(input_shape=INPUT_SHAPE,
                            class_num=CLASS_NUM,
                            last_layer=LAST_LAYER,
                            dense_nodes=1024,
                            lr=0.01,
                            dropout=0.2,
                            loss='categorical_crossentropy',
                            metrics=['accuracy'],
                            print_summary = False)


Loading Inception-v3 weights from:
	 PATH = models/raw/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5...
Setting last layer as: mixed8
Last layer output shape: (None, 6, 6, 1280)
Initializing Dense layers...
Building Dense layers...
	Adding Dense layer with 1024 nodes
	Adding classification layer for 5 classes
Compiling the model...
Model built in: 3.4980528354644775 seconds 



The code to compile the inception-v3 model has succesfully been formatted into a resuable format to quickly make new models when training. 